In [1]:
# import cv2
# import os

# video_path = 'cow-images/' ## put in new video to train in this folder
# output_dir = 'data/images/test'  

# os.makedirs(output_dir, exist_ok=True)

# vidcap = cv2.VideoCapture(video_path)
# success, image = vidcap.read()
# count = 0
# frame_rate = 500  

# while success:
#     if count % frame_rate == 0:
#         frame_filename = os.path.join(output_dir, f'frame_{count}.jpg')
#         cv2.imwrite(frame_filename, image)  
#     success, image = vidcap.read()
#     count += 1

# vidcap.release()


In [28]:
!python train.py --img 640 --batch 16 --epochs 100 --data data/cow_ear_tags.yaml --weights yolov5s.pt


github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5



train: weights=yolov5s.pt, cfg=, data=data/cow_ear_tags.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
YOLOv5  2024-5-27 Python-3.10.6 torch-2.3.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

In [1]:
!python detect.py --weights runs/train/exp14/weights/best.pt --img 640 --conf 0.25 --source data/images/test/test17.JPEG




detect: weights=['runs/train/exp14/weights/best.pt'], source=data/images/test/test17.JPEG, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  4000527f Python-3.10.6 torch-2.3.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\steve\Downloads\dap_thewi\dap_thewi-main\data\images\test\test17.JPEG: 640x480 1 ear_tag, 107.1ms
Speed: 2.3ms pre-process, 107.1ms inference, 9.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp47


In [16]:
# from PIL import Image
# import matplotlib.pyplot as plt
# import os

# result_dir = 'runs/detect/exp43'

# result_image_path = os.path.join(result_dir, 'calf1.jpeg')

# result_image = Image.open(result_image_path)

# plt.imshow(result_image)
# plt.axis('off')  
# plt.show()



In [ ]:
# !python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source data/images/test/test_vid.MP4


In [2]:
import os
import cv2
import torch
import easyocr
import pandas as pd
from Levenshtein import distance as levenshtein_distance
import matplotlib.pyplot as plt

# This loads the trained weight file to detect the tags, mine is exp14, you need to change it to your most recent one oleksandr

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp14/weights/best.pt', force_reload=True)

# Extracting the bounding boxes (so the annotatated boxes)
def get_bounding_boxes(image):
    results = model(image)
    boxes = results.xyxy[0].numpy()  # Get tge bounding box coordinates
    bounding_boxes = []

    for box in boxes:
        x_min, y_min, x_max, y_max, confidence, class_id = box
        if confidence > 0.5:  # Filter out low-confidence detections, i just put it at min 0.5, you can adjust to make it more picky
            bounding_boxes.append((int(x_min), int(y_min), int(x_max), int(y_max)))

    return bounding_boxes

# This is the function to find the best match from the I&R dataset
def find_best_match(detected_text, ir_numbers):
    best_match = None
    min_distance = float('inf')
    for number in ir_numbers:
        distance = levenshtein_distance(detected_text, number)
        if distance < min_distance:
            min_distance = distance
            best_match = number
    return best_match

# Generate all possible orders of detected text segments, to best find the different sequences that the text on the ear tah can be in
def generate_permutations(text_segments):
    if len(text_segments) == 0:
        return ['']
    permutations = []
    for i in range(len(text_segments)):
        rest = text_segments[:i] + text_segments[i+1:]
        for perm in generate_permutations(rest):
            permutations.append(text_segments[i] + perm)
    return permutations

# Load the I&R dataset from the I&R dataset folder
ir_dataset_path = 'I&R datasets/I&R Verhagen group MAY.xls'
ir_data = pd.read_excel(ir_dataset_path, sheet_name=0, usecols='B', skiprows=7)  
ir_numbers = ir_data.iloc[:, 0].astype(str).tolist()

# Load the image (from the exp folder that was used to store the the test image)
image_path = 'runs/detect/exp46/test17.jpeg'
image = cv2.imread(image_path)

# Get bounding box coordinates
bounding_boxes = get_bounding_boxes(image)

# Initialize EasyOCR Reader
reader = easyocr.Reader(['en', 'nl', 'de'])  # English, Dutch, German

# Directory to save the cropped images
save_dir = 'runs/detect/exp46'
os.makedirs(save_dir, exist_ok=True)

# File to save bounding box coordinates and detected text
bbox_file_path = os.path.join(save_dir, 'bounding_boxes.txt')

with open(bbox_file_path, 'w') as bbox_file:
    for i, (x_min, y_min, x_max, y_max) in enumerate(bounding_boxes):
        # Write bounding box coordinates to file
        bbox_file.write(f'Bounding Box {i}: ({x_min}, {y_min}, {x_max}, {y_max})\n')

        # Crop the image using bounding box coordinates
        cropped_image = image[y_min:y_max, x_min:x_max]
        
        # preprocess the cropped image (e.g., converting to grayscale) so it might be more readable
        cropped_image_gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        
        # Use EasyOCR to read text from the cropped image
        result = reader.readtext(cropped_image_gray)
        
        # Concatenate detected text from each cropped image
        detected_text = ' '.join([text for (bbox, text, prob) in result])
        print(f"Detected text: {detected_text}")

       # Generate permutations of detected text
        detected_text_segments = detected_text.split()
        permutations = generate_permutations(detected_text_segments)

         # Find the best match from the I&R dataset
        best_match = None
        min_distance = float('inf')
        for perm in permutations:
            match = find_best_match(perm.replace(' ', ''), ir_numbers)
            distance = levenshtein_distance(perm.replace(' ', ''), match)
            if distance < min_distance:
                min_distance = distance
                best_match = match
        print(f"Best match from I&R dataset: {best_match}")

        # Save detected text and best match to the file
        bbox_file.write(f"Detected text: {detected_text}\n")
        bbox_file.write(f"Best match from I&R dataset: {best_match}\n")

        # Save the cropped image
        cropped_image_path = os.path.join(save_dir, f'cropped_image_{i}.jpg')
        cv2.imwrite(cropped_image_path, cropped_image)
        print(f'Cropped image saved to {cropped_image_path}')

    


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\steve/.cache\torch\hub\master.zip
YOLOv5  2024-6-13 Python-3.10.6 torch-2.3.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Detected text: DE 01254 87185 NL AA
Best match from I&R dataset: DE0125487185
Cropped image saved to runs/detect/exp46\cropped_image_0.jpg
